# Sandbox 

Sandbox for outputting to csv

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
data_folder = Path("../data/")

In [ ]:
# Concatenate all files in the 2_morphometrics folder
morpho_folder = data_folder / "2_morphometrics"
csv_folder = data_folder / "4_csv"
out_csv = csv_folder / "Morphometrics.csv"
df = pd.read_csv(out_csv)

if df["city"].nunique() > len(city_list):
    logger.info("CSV exists and has more cities. Skipping.")
    return

df_full = pd.DataFrame()
for file in morpho_folder.iterdir():
    if " - morpho.gpkg" in file.name:
        city_name = file.stem.split(" - morpho")[0].strip()
        gdf = gpd.read_file(file)
        gdf["city"] = city_name
        df_full = pd.concat([df_full, gdf])

# Save
df_full.to_csv(out_csv, index=None)
logger.info("CSV: Saved %s", out_csv)

# Generate tests

In [18]:
import sys
sys.path.append("..")
from pathlib import Path
import geopandas as gpd
import osmnx as ox

import networkx as nx
from morpho.layers import clean_gdf, get_graph, get_polygons